In [2]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math
import warnings

In [3]:
# Assumptions
thresholds = [300, 500, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)
park_entry_point_merge = 17.5

In [4]:
cities = ['Philadelphia, United States','Denver, United States','Ghent, Belgium','Amsterdam, Netherlands',
          'Dhaka Metropolitan, Bangladesh']

thresholds
# 'Dublin, Ireland' is problematic; road system is too dense.

[300, 500, 1000]

In [5]:
warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()
roads = list()

for i in range(len(cities)):
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(cities[i], network_type="all", buffer_dist = (np.max(thresholds)+1000))
    graphs.append(graph)
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_nodes.append(road_node)
    road = road_node.reset_index()
    roads.append(road)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    road_edges.append(road_edge)
    
    # Get only necessary road connections columns for network performance
    road_con = road_edge[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    print(cities[i].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
road_edges[0]

Philadelphia done 2.17 mns
Denver done 5.31 mns
Ghent done 5.97 mns
Amsterdam done 7.5 mns
Dhaka Metropolitan done 8.29 mns


,from,to,key,osmid,name,highway,oneway,length,geometry,ref,lanes,width,bridge,service,tunnel,maxspeed,access,junction,area,geometry_m
0,103237949,103353127,103237949-103353127,11591597,New Jersey Avenue,residential,False,111.799,"LINESTRING (-74.96444 40.03625, -74.96557 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689617..."
1,103237949,103353090,103237949-103353090,11591597,New Jersey Avenue,residential,False,107.489,"LINESTRING (-74.96444 40.03625, -74.96337 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689364..."
2,103237949,103237976,103237949-103237976,11580386,Cleveland Avenue,residential,False,168.717,"LINESTRING (-74.96444 40.03625, -74.96404 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689552..."
3,103237976,103238007,103237976-103238007,11580386,Cleveland Avenue,residential,False,151.349,"LINESTRING (-74.96340 40.03496, -74.96245 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689797..."
4,103237976,103590312,103237976-103590312,11610261,2nd Street,residential,False,108.508,"LINESTRING (-74.96340 40.03496, -74.96448 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689779..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338097,9726316545,5550941352,9726316545-5550941352,579660432,NaN,service,False,39.965,"LINESTRING (-75.24128 40.06507, -75.24111 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695935..."
338098,9726316545,9726316548,9726316545-9726316548,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24128 40.06507, -75.24148 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695962..."
338099,9726316548,9726316545,9726316548-9726316545,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24175 40.06470, -75.24175 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5696003.982 8483808.258, -5695998..."
338100,9728655691,110122187,9728655691-110122187,12149073,South 10th Street,residential,True,3.250,"LINESTRING (-75.15829 39.94437, -75.15830 39.9...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5711864.888 8467388.116, -5711869..."


In [6]:
# Get city boundaries
warnings.filterwarnings("ignore")
bound_df = pd.DataFrame()
for i in range(len(cities)):
    bound_df = pd.concat([bound_df, pd.DataFrame(ox.geocoder.geocode_to_gdf(cities[i]))])
    
bound_df = bound_df.reset_index()
bound_df = bound_df.loc[:,bound_df.columns!='index']
bound_df = gpd.GeoDataFrame(bound_df, geometry = 'geometry', crs = 4326)
bound_df.to_file(r'C:\Users\bartb\Downloads\bounds.shp')
bound_df

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...",40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,40.002410,-75.139364,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797
1,"POLYGON ((-105.10988 39.62710, -105.10761 39.6...",39.914209,39.614315,-104.599700,-105.109885,327393456,relation,1411339,39.739236,-104.984862,"Denver, Colorado, United States",boundary,administrative,0.905476
2,"POLYGON ((3.57976 51.03027, 3.57992 51.03022, ...",51.188886,50.979542,3.849325,3.579762,282032351,relation,897671,51.084199,3.732851,"Ghent, Gent, East Flanders, Flanders, Belgium",boundary,administrative,0.855558
3,"MULTIPOLYGON (((4.72876 52.40071, 4.73371 52.4...",52.431064,52.278174,5.079162,4.728756,282057196,relation,271110,52.372760,4.893604,"Amsterdam, North Holland, Netherlands",boundary,administrative,0.946813
4,"POLYGON ((90.32978 23.75081, 90.32981 23.75050...",23.899531,23.668077,90.508832,90.329784,328693247,relation,13663697,23.783663,90.403246,"Dhaka Metropolitan, Dhaka District, Dhaka Divi...",boundary,administrative,0.710000


In [7]:
# Get the grids
grids = list()
for i in range(len(cities)):
    # extract grids
    path = 'D:/Dumps/Population_grids/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_CPoPGrid.gpkg'
    file = gpd.read_file(path).to_crs(4326)

    # Get grid and boundary overlay
    popgrid = file.overlay(bound_df.iloc[i:i+1])

    # Only get full grids (area differs slightly due to lon-lat placement) and grids with people in it.
    popgrid['area'] = popgrid.area / popgrid.area.max()
    popgrid = popgrid[(popgrid['area'] >= 0.99)]
    popgrid = popgrid[popgrid['PoP2015_Number'] > 0]
    
    # Get grids centroids and lon-lat
    popgrid['centroid'] = popgrid.to_crs(4326).centroid
    popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    
    popgrid = popgrid.reset_index()
    grids.append(popgrid)
grids[0]

,index,grid_id,PoP2015_Number,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,...,display_name,class,type,importance,geometry,area,centroid,centroid_m,grid_lon,grid_lat
0,6,259,113,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",0.996208,POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),-5.675978e+06,8.459813e+06
1,7,260,135,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",0.996208,POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),-5.675889e+06,8.459539e+06
2,13,400,137,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",0.996234,POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),-5.676341e+06,8.459996e+06
3,14,401,104,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01452 40.13349, -75.01452 40.131...",0.996234,POINT (-75.01564 40.13262),POINT (-5676251.742 8459722.980),-5.676252e+06,8.459723e+06
4,15,402,116,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01227 40.13349, -75.01227 40.131...",0.996234,POINT (-75.01339 40.13262),POINT (-5676162.159 8459449.641),-5.676162e+06,8.459450e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7868,10279,21026,22,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25482 39.88135, -75.25482 39.879...",0.999949,POINT (-75.25594 39.88048),POINT (-5725941.707 8476084.513),-5.725942e+06,8.476085e+06
7869,10280,21027,22,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25257 39.88135, -75.25257 39.879...",0.999949,POINT (-75.25369 39.88048),POINT (-5725852.550 8475808.155),-5.725853e+06,8.475808e+06
7870,10281,21028,2,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25033 39.88135, -75.25033 39.879...",0.999949,POINT (-75.25145 39.88048),POINT (-5725763.377 8475531.806),-5.725763e+06,8.475532e+06
7871,10313,21166,6,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25931 39.87962, -75.25931 39.877...",0.999975,POINT (-75.26043 39.87875),POINT (-5726396.363 8476548.128),-5.726396e+06,8.476548e+06


In [8]:
parks_in_range = list()
for i in range(len(cities)):
    path = 'D:/Dumps/Greenspace/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_Greenspace.gpkg'
    greenspace = gpd.read_file(path).to_crs(4326)

    warnings.filterwarnings("ignore")

    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')

    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
        
    # Get the park buffer
    parks_t = pd.DataFrame(parks.index).set_index('components')
    parks_t['park_geom'] = parks['geometry']
    parks_t = gpd.GeoDataFrame(parks_t, geometry = 'park_geom', crs = 4326)
    for j in range(len(thresholds)):
        Bound_buffer = gpd.GeoDataFrame(
            geometry = np.repeat(bound_df.iloc[i:i+1].to_crs(3043).
                                 buffer(thresholds[j]).to_crs(4326),len(parks)), 
            crs = 4326).reset_index().iloc[:,1]
        Bound_buffer.index = parks.index
        parks_t = parks_t.join(Bound_buffer, how = 'outer')
        parks_t = parks_t.rename(columns = {'geometry':('buffer_' + str(thresholds[j]))})
        
        # Check which parks intersect with the city boundary with buffer.
        parks_within_range = parks_t[parks_t['park_geom'].intersection(parks_t[('buffer_' + str(thresholds[j]))]).area > 0]
        pwr = pd.DataFrame(parks_within_range.index).set_index('components')
        pwr[str(thresholds[j])] = 1
        parks_t = parks_t.join(pwr, how = 'left')
        parks_t[str(thresholds[j])] = parks_t[str(thresholds[j])].fillna(0)
    parks_t = parks_t.join(parks)
    
    thresholds_str = list()
    for k in range(len(thresholds)): thresholds_str.append(str(thresholds[k]))
    parks_t = parks_t[(parks_t[thresholds_str] == 1).any(axis = 1)]
    parks_t = parks_t.reset_index()
    parks_t['geometry_m'] = parks_t['geometry'].to_crs(3043)
    parks_in_range.append(parks_t)
    
parks_in_range[0]

,components,park_geom,buffer_300,300,buffer_500,500,buffer_1000,1000,geometry,OBJECTID,...,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,geometry_m
0,5,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,...,2020,1983,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061...","MULTIPOLYGON (((-5698736.789 8484341.822, -569..."
1,6,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,...,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916...","POLYGON ((-5719765.167 8476866.315, -5719770.3..."
2,7,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,...,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921...","MULTIPOLYGON (((-5714730.553 8479503.851, -571..."
3,8,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,...,2020,1978,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961...","POLYGON ((-5708727.262 8467262.277, -5708788.5..."
4,9,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,...,2020,None,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033...","MULTIPOLYGON (((-5698195.124 8473695.244, -569..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,734,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",0.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",0.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,...,2020,None,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949...","POLYGON ((-5714991.814 8479820.456, -5715022.3..."
469,735,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,...,2020,None,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968...","POLYGON ((-5712611.388 8483038.861, -5712651.7..."
470,736,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",0.0,"POLYGON ((-75.24982 39.87326, -75.25104 39.873...",0.0,"POL

In [9]:
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['park_geom','geometry']], left_on = 'Park_No', right_index = True)
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][thresholds_str], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_proot2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_proot4'] = ParkRoad['size_infl_factor']**(1/4)    
    ParkRoads.append(ParkRoad)
    
    print(cities[j].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')

ParkRoads[0]

Philadelphia 0.0 % done 0.01  mns
Philadelphia 10.6 % done 1.02  mns
Philadelphia 21.1 % done 1.75  mns
Philadelphia 31.7 % done 2.1  mns
Philadelphia 42.3 % done 2.49  mns
Philadelphia 52.9 % done 2.82  mns
Philadelphia 63.4 % done 3.12  mns
Philadelphia 74.0 % done 3.5  mns
Philadelphia 84.6 % done 3.86  mns
Philadelphia 95.1 % done 4.25  mns
Philadelphia done 4.51 mns
Denver 0.0 % done 4.82  mns
Denver 14.7 % done 7.93  mns
Denver 29.3 % done 9.34  mns
Denver 44.0 % done 11.4  mns
Denver 58.7 % done 13.49  mns
Denver 73.3 % done 14.08  mns
Denver 88.0 % done 14.68  mns
Denver done 15.24 mns
Ghent 0.0 % done 15.24  mns
Ghent 40.3 % done 15.42  mns
Ghent 80.6 % done 15.61  mns
Ghent done 15.71 mns
Amsterdam 0.0 % done 15.72  mns
Amsterdam 64.9 % done 16.06  mns
Amsterdam done 16.23 mns
Dhaka Metropolitan 0.0 % done 16.23  mns
Dhaka Metropolitan 29.8 % done 16.34  mns
Dhaka Metropolitan 59.5 % done 16.44  mns
Dhaka Metropolitan 89.3 % done 16.54  mns
Dhaka Metropolitan done 16.58 mns


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,Park_No,park_lon,...,300,500,1000,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot4
0,109991185,40.053816,-75.240687,3,NaN,NaN,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,...,1.0,1.0,1.0,"POLYGON ((-75.23914 40.05656, -75.23880 40.056...",198723.068185,4.192678e+06,0.047398,1.990313,1.410785,1.187765
1,109991191,40.054803,-75.239081,3,NaN,NaN,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,...,1.0,1.0,1.0,"POLYGON ((-75.24278 40.05572, -75.24265 40.055...",291442.444263,4.192678e+06,0.069512,2.918945,1.708492,1.307093
2,109991199,40.055776,-75.237501,3,NaN,NaN,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,...,1.0,1.0,1.0,"POLYGON ((-75.24077 40.05416, -75.24096 40.054...",245064.061026,4.192678e+06,0.058450,2.454442,1.566666,1.251665
3,109991204,40.056232,-75.236783,3,NaN,NaN,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,...,1.0,1.0,1.0,"POLYGON ((-75.24005 40.05461, -75.24024 40.054...",187511.352028,4.192678e+06,0.044724,1.878022,1.370409,1.170645
4,110169862,40.063972,-75.249174,3,NaN,NaN,POINT (-75.24917 40.06397),POINT (-5696411.964 8484679.155),0,-5.696412e+06,...,1.0,1.0,1.0,"POLYGON ((-75.24676 40.06163, -75.24707 40.061...",194283.946127,4.192678e+06,0.046339,1.945853,1.394938,1.181075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,3425873720,39.966878,-75.277524,3,NaN,NaN,POINT (-75.27752 39.96688),POINT (-5713004.185 8483178.234),471,-5.713004e+06,...,0.0,1.0,1.0,"POLYGON ((-75.27647 39.96684, -75.27650 39.966...",19734.719624,1.973472e+04,1.000000,0.197653,0.444582,0.666770
7316,110006563,39.961942,-75.273003,3,NaN,NaN,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),472,-5.713613e+06,...,0.0,0.0,1.0,"POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.829085
7317,110133414,39.961600,-75.274141,3,NaN,NaN,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),472,-5.713713e+06,...,0.0,0.0,1.0,"POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.829085
7318,923789083,39.962719,-75.274196,3,NaN,NaN,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),472,-5.713536e+06,...,0.0,0.0,1.0,"POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.829085


In [10]:
# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    print(1)
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    print(2)
    ParkComb['components'] = M.component_labels
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

1
2
1
2
1
2
1
2
1
2


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,Park_No,park_lon,...,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot4,geometry_m_buffer,components,comp_centroid,centr_dist
0,109991185,40.053816,-75.240687,3,NaN,NaN,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,...,198723.068185,4.192678e+06,0.047398,1.990313,1.410785,1.187765,"POLYGON ((-5697677.534 8483120.448, -5697677.6...",0,POINT (-75.24069 40.05382),0.0
1,109991191,40.054803,-75.239081,3,NaN,NaN,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,...,291442.444263,4.192678e+06,0.069512,2.918945,1.708492,1.307093,"POLYGON ((-5697457.112 8482974.522, -5697457.1...",1,POINT (-75.23908 40.05480),0.0
2,109991199,40.055776,-75.237501,3,NaN,NaN,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,...,245064.061026,4.192678e+06,0.058450,2.454442,1.566666,1.251665,"POLYGON ((-5697239.944 8482831.074, -5697240.0...",2,POINT (-75.23750 40.05578),0.0
3,109991204,40.056232,-75.236783,3,NaN,NaN,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,...,187511.352028,4.192678e+06,0.044724,1.878022,1.370409,1.170645,"POLYGON ((-5697139.054 8482766.601, -5697139.1...",3,POINT (-75.23678 40.05623),0.0
4,110169862,40.063972,-75.249174,3,NaN,NaN,POINT (-75.24917 40.06397),POINT (-5696411.964 8484679.155),0,-5.696412e+06,...,194283.946127,4.192678e+06,0.046339,1.945853,1.394938,1.181075,"POLYGON ((-5696394.464 8484679.155, -5696394.5...",4,POINT (-75.24917 40.06397),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,3425873720,39.966878,-75.277524,3,NaN,NaN,POINT (-75.27752 39.96688),POINT (-5713004.185 8483178.234),471,-5.713004e+06,...,19734.719624,1.973472e+04,1.000000,0.197653,0.444582,0.666770,"POLYGON ((-5712986.685 8483178.234, -5712986.7...",4000,POINT (-75.27752 39.96688),0.0
7316,110006563,39.961942,-75.273003,3,NaN,NaN,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),472,-5.713613e+06,...,47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.829085,"POLYGON ((-5713595.774 8482370.035, -5713595.8...",4001,POINT (-75.27300 39.96194),0.0
7317,110133414,39.961600,-75.274141,3,NaN,NaN,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),472,-5.713713e+06,...,47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.829085,"POLYGON ((-5713695.236 8482492.281, -5713695.3...",4002,POINT (-75.27414 39.96160),0.0
7318,923789083,39.962719,-75.274196,3,NaN,NaN,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),472,-5.713536e+06,...,47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.829085,"POLYGON ((-5713518.906 8482556.428, -5713518.9...",4003,POINT (-75.27420 39.96272),0.0


In [11]:
start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    print(cities[l])
    # Check all parks within 1000m radius
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    len3 = int(np.ceil(len2/1000))
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per 1000
        l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_id','grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_factor','size_infl_proot2','size_infl_proot4','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl = list(node_merged['size_infl_factor'])
        infl2 = list(node_merged['size_infl_proot2'])
        infl4 = list(node_merged['size_infl_proot4'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl4[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl','size_infl_eucl4']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])

        print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within a one of',thresholds,'m threshold:',len(mat_df))

        # Checks the number of the parks within 1000m.
        len_mat = len_mat + len(mat_df)

    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl','size_infl_eucl4',
                      'Grid_No','Park_entry_No','Grid_Id','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_factor','size_infl_proot2','size_infl_proot4',
                      'parkshare_walked','park_area','walk_area_m2']
    output = output[['raw euclidean','size_infl_eucl','size_infl_eucl2','size_infl_eucl4',
                     'Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_factor','size_infl_proot2','size_infl_proot4']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        nearest = int(roads[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
        mat5.append(roads[l]['osmid'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
                                                   ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]


Philadelphia, United States
1 / 5 comb. done 0.5  mns
of 7873000 within a one of [300, 500, 1000] m threshold: 95107
2 / 5 comb. done 0.88  mns
of 7873000 within a one of [300, 500, 1000] m threshold: 53429
3 / 5 comb. done 1.39  mns
of 7873000 within a one of [300, 500, 1000] m threshold: 106062
4 / 5 comb. done 1.96  mns
of 7873000 within a one of [300, 500, 1000] m threshold: 104442
5 / 5 comb. done 2.46  mns
of 5000 within a one of [300, 500, 1000] m threshold: 21419
total combinations within distance 380459
0.0 % gridentry done 2.46  mns
65.7 % gridentry done 2.89  mns
100 % gridentry done 3.1  mns
Denver, United States
1 / 4 comb. done 3.58  mns
of 6181000 within a one of [300, 500, 1000] m threshold: 96459
2 / 4 comb. done 4.06  mns
of 6181000 within a one of [300, 500, 1000] m threshold: 89489
3 / 4 comb. done 4.39  mns
of 6181000 within a one of [300, 500, 1000] m threshold: 32588
4 / 4 comb. done 4.68  mns
of 887000 within a one of [300, 500, 1000] m threshold: 52931
total co

,index,raw euclidean,size_infl_eucl,size_infl_eucl2,size_infl_eucl4,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,...,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_factor,size_infl_proot2,size_infl_proot4,grid_osm,geometry,geometry_m,grid_entry_dist
0,1611,1944.923698,977.194857,1378.611415,1637.465729,1611,0,5838,POINT (-75.24247 40.06536),POINT (-5695926.453 8483929.676),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.990313,1.410785,1.187765,9726316548,POINT (-75.24175 40.06470),POINT (-5696003.982 8483808.258),144.059
1,9484,1819.097111,623.203626,1064.738426,1391.712109,1611,1,5838,POINT (-75.24247 40.06536),POINT (-5695926.453 8483929.676),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697474.612 8482974.522),291442.444263,2.918945,1.708492,1.307093,9726316548,POINT (-75.24175 40.06470),POINT (-5696003.982 8483808.258),144.059
2,17357,1725.822793,703.142684,1101.589611,1378.821402,1611,2,5838,POINT (-75.24247 40.06536),POINT (-5695926.453 8483929.676),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697257.444 8482831.074),245064.061026,2.454442,1.566666,1.251665,9726316548,POINT (-75.24175 40.06470),POINT (-5696003.982 8483808.258),144.059
3,25230,1692.894531,901.424224,1235.320258,1446.121333,1611,3,5838,POINT (-75.24247 40.06536),POINT (-5695926.453 8483929.676),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697156.554 8482766.601),187511.352028,1.878022,1.370409,1.170645,9726316548,POINT (-75.24175 40.06470),POINT (-5696003.982 8483808.258),144.059
4,33103,892.994936,458.922096,640.168031,756.086510,1611,4,5838,POINT (-75.24247 40.06536),POINT (-5695926.453 8483929.676),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5696411.964 8484679.155),194283.946127,1.945853,1.394938,1.181075,9726316548,POINT (-75.24175 40.06470),POINT (-5696003.982 8483808.258),144.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380454,3832226,714.062022,7622.702956,2333.041510,1290.711563,5948,4929,13133,POINT (-75.12344 39.97730),POINT (-5705229.396 8464822.317),...,"POLYGON ((-75.12794 39.97893, -75.12794 39.978...",POINT (-5705009.525 8465501.685),9353.062299,0.093676,0.306065,0.553231,110191705,POINT (-75.12333 39.97751),POINT (-5705191.956 8464820.384),37.489
380455,3840099,593.797486,6338.863733,1940.103438,1073.325927,5948,4930,13133,POINT (-75.12344 39.97730),POINT (-5705229.396 8464822.317),...,"POLYGON ((-75.12794 39.97893, -75.12794 39.978...",POINT (-5705088.289 8465399.105),9353.062299,0.093676,0.306065,0.553231,110191705,POINT (-75.12333 39.97751),POINT (-5705191.956 8464820.384),37.489
380456,3832227,963.270568,10283.035901,3147.275939,1741.171526,5949,4929,13134,POINT (-75.12119 39.97730),POINT (-5705139.805 8464547.265),...,"POLYGON ((-75.12794 39.97893, -75.12794 39.978...",POINT (-5705009.525 8465501.685),9353.062299,0.093676,0.306065,0.553231,110165097,POINT (-75.12163 39.97749),POINT (-5705127.165 8464610.652),64.635
380457,3840100,853.395631,9110.106969,2788.283610,1542.565737,5949,4930,13134,POINT (-75.12119 39.97730),POINT (-5705139.805 8464547.265),...,"POLYGON ((-75.12794 39.97893, -75.12794 39.978...",POINT (-5705088.289 8465399.105),9353.062299,0.093676,0.306065,0.553231,110165097,POINT (-75.12163 39.97749),POINT (-5705127.165 8464610.652),64.635


In [12]:
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Philadelphia, United States 380459
Denver, United States 271467
Ghent, Belgium 353675
Amsterdam, Netherlands 113194
Dhaka Metropolitan, Bangladesh 27926


In [13]:
warnings.filterwarnings("ignore")

start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j]

    parknode = list(CityRoads['Parkroad_osmid'])
    gridnode = list(CityRoads['grid_osm'])

    s_mat = list([])
    s_mat1 = list([])
    s_mat2 = list([])
    s_mat3 = list([])
    s_mat4 = list([])

    for i in range(len(CityRoads)):
        # First try from population grid to park Dijkstra algorithm
        try:
            shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
            s_mat.append(shortest)
            shortest_to = list(shortest[1:len(shortest)])
            shortest_to.append(0)
            s_mat1.append(shortest_to)
            s_mat2.append(list(np.repeat(i, len(shortest))))
            s_mat3.append(list(np.arange(0, len(shortest))))
            s_mat4.append('normal way')

        except:
            try:
                # Check the reverse
                shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('reverse way')
            except:
                try:
                    # Get the route to the nearest park node
                    new_node = pd.DataFrame((abs(CityRoads['Parkroad_coords_centroid'].x[i] - CityRoads['geometry_y'].x)**2
                                             + abs(CityRoads['Parkroad_coords_centroid'].y[i] - CityRoads['geometry_y'].y)**2
                                            )**(1/2)).join(CityRoads['osmid']).sort_values(0).iloc[1,1]
                    shortest = nx.shortest_path(Graph, gridnode[i], new_node, 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(0)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('nearest way')
                    print('on ',i,' nearest node used')
                except:
                    # Print if none of the above three options gives any result
                    print('index',i,'no route possible between osmid',gridnode[i],'and',parknode[i])
                    pass

        if i % 10000 == 0: print(cities[j].rsplit(',')[0], 
                                round(i/len(CityRoads)*100,2),'% done', round((time.time() - start_time) / 60,2),'mns')
    print(cities[j].rsplit(',')[0], '100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')
    
    # Unpack lists
    s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
    s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
    s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
    s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

    # Format df
    routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
    routes.columns = ['from','to','route','step']
    mat_key = list([])
    for i in range(len(routes)):
        mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
    routes['key'] = mat_key
    routes = routes.set_index('key')

    # Add route information
    routes = routes.join(road_conn[j], how = 'left')
    routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
    routes.sort_values(by = ['route','step'])
    print(cities[j].rsplit(',')[0], 'formatting done', round((time.time() - start_time) / 60,2), 'mns')
    
    # get single (dissolved) line per route, attach information.
    routes2 = routes[['route','geometry']].dissolve('route')
    routes2['way_calculated'] = s_mat4
    routes2['route_cost'] = routes.groupby('route')['length'].sum()
    routes2['num_steps'] = routes.groupby('route')['step'].max()
    routes2.index = routes2.index.astype(int)
    routes2 = pd.merge(routes2, CityRoads[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm',
                                          'walk_area_m2','size_infl_factor','size_infl_proot2', 'size_infl_proot4',
                                          'raw euclidean']],
                       left_index = True, right_index = True)
    routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
    routes2['grav_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_factor']
    routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
    routes2['grav4_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot4']

    routes2['parkcost_ha'] = routes2['grav_total_cost'] * (routes2['walk_area_m2'] /10000)
    routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
    
    print(cities[j].rsplit(',')[0], 'dissolving done', round((time.time() - start_time) / 60,2), 'mns')
    Routes.append(routes2)
Routes[0]

Philadelphia 0.0 % done 0.0 mns
Philadelphia 2.63 % done 0.27 mns
Philadelphia 5.26 % done 0.59 mns
Philadelphia 7.89 % done 0.99 mns
Philadelphia 10.51 % done 2.21 mns
Philadelphia 13.14 % done 2.97 mns
Philadelphia 15.77 % done 3.84 mns
Philadelphia 18.4 % done 5.09 mns
Philadelphia 21.03 % done 6.16 mns
Philadelphia 23.66 % done 6.83 mns
Philadelphia 26.28 % done 7.81 mns
Philadelphia 28.91 % done 8.61 mns
Philadelphia 31.54 % done 9.11 mns
Philadelphia 34.17 % done 9.71 mns
Philadelphia 36.8 % done 10.29 mns
Philadelphia 39.43 % done 11.13 mns
Philadelphia 42.05 % done 11.98 mns
Philadelphia 44.68 % done 12.66 mns
Philadelphia 47.31 % done 13.23 mns
Philadelphia 49.94 % done 14.3 mns
Philadelphia 52.57 % done 15.4 mns
Philadelphia 55.2 % done 16.26 mns
Philadelphia 57.82 % done 17.08 mns
Philadelphia 60.45 % done 17.61 mns
Philadelphia 63.08 % done 18.53 mns
Philadelphia 65.71 % done 19.4 mns
Philadelphia 68.34 % done 19.98 mns
Philadelphia 70.97 % done 20.85 mns
Philadelphia 73.6 

,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,...,size_infl_factor,size_infl_proot2,size_infl_proot4,raw euclidean,raw_total_cost,grav_total_cost,grav2_total_cost,grav4_total_cost,parkcost_ha,gridpark_no
route,,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,2360.760,20,1611,0,0,144.059,109991185,9726316548,...,1.990313,1.410785,1.187765,1944.923698,2504.819,1258.505023,1775.479454,2108.851505,25009.397947,1611-0
1,"MULTILINESTRING ((-75.24053 40.06186, -75.2412...",normal way,1738.625,20,1611,0,1,144.059,109991191,9726316548,...,2.918945,1.708492,1.307093,1819.097111,1882.684,644.987829,1101.956562,1440.359673,18797.682932,1611-0
2,"MULTILINESTRING ((-75.24053 40.06186, -75.2402...",normal way,1556.118,19,1611,0,2,144.059,109991199,9726316548,...,2.454442,1.566666,1.251665,1725.822793,1700.177,692.693957,1085.219947,1358.332063,16975.439413,1611-0
3,"MULTILINESTRING ((-75.24053 40.06186, -75.2412...",normal way,1486.623,18,1611,0,3,144.059,109991204,9726316548,...,1.878022,1.370409,1.170645,1692.894531,1630.682,868.297599,1189.923218,1392.977521,16281.565679,1611-0
4,"MULTILINESTRING ((-75.24009 40.06734, -75.2402...",normal way,1573.117,15,1611,0,4,144.059,110169862,9726316548,...,1.945853,1.394938,1.181075,892.994936,1717.176,882.479818,1231.004941,1453.909261,17145.166150,1611-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380454,"MULTILINESTRING ((-75.12734 39.98013, -75.1279...",normal way,615.082,7,5948,215,4929,37.489,109837023,110191705,...,0.093676,0.306065,0.553231,714.062022,652.571,6966.278471,2132.133042,1179.562712,6515.603654,5948-215
380455,"MULTILINESTRING ((-75.12677 39.97851, -75.1273...",normal way,651.644,8,5948,215,4930,37.489,109837035,110191705,...,0.093676,0.306065,0.553231,593.797486,689.133,7356.582474,2251.591382,1245.650803,6880.657419,5948-215
380456,"MULTILINESTRING ((-75.12734 39.98013, -75.1279...",normal way,816.802,10,5949,215,4929,64.635,109837023,110165097,...,0.093676,0.306065,0.553231,963.270568,881.437,9409.452147,2879.902650,1593.252256,8800.719213,5949-215


In [23]:
# get from routes to population score in high, medium, low and no.

start_time = time.time()
popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav','grav2','grav4'])
    m1 = list(['entrance','gravity','gravity**(1/2)','gravity**(1/4)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Routes[n].iloc[Routes[n].groupby('gridpark_no')[(str(l1[i]) +'_total_cost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_total_cost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['PoP2015_Number','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_total_cost'] <= t]
            thold[score] = t - thold[l1[i] + '_total_cost']
            thold['pop' + score] = thold[score] * thold['PoP2015_Number']
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,'walk_area_ha' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,'walkha_' + score]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score,'walk_area_ha' + str2].sum()
            gs[score + '_parks'] = locals()[str1].groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(locals()[str1][['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = locals()[str1][locals()[str1]['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + locals()[str1][locals()[str1]['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])

            gs = gs.join(grids[n]['PoP2015_Number'])
            grdsc = pd.concat([grdsc, gs], axis = 1)

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['PoP2015_Number'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            print('grid ',t)

            grid_score.append(grdsc)

        grid_scores.append(grid_score)
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        grdsc.to_csv('C:/Users/bartb/Downloads/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
    
    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
popg_acc    
# Below

Exception ignored in: <function BaseGeometry.__del__ at 0x00000208909AAAF0>
Traceback (most recent call last):
  File "C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\shapely\geometry\base.py", line 209, in __del__
    self._empty(val=None)
  File "C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\shapely\geometry\base.py", line 194, in _empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 


Philadelphia, United States
entrance 3.49 mns
grid  300
grid  500
grid  1000
gravity 4.33 mns
grid  300
grid  500
grid  1000
gravity**(1/2) 4.72 mns
grid  300
grid  500
grid  1000
gravity**(1/4) 5.09 mns
grid  300
grid  500
grid  1000
Philadelphia, United States done 5.42 mns
Denver, United States
entrance 5.46 mns
grid  300
grid  500
grid  1000
gravity 6.19 mns
grid  300
grid  500
grid  1000
gravity**(1/2) 6.59 mns
grid  300
grid  500
grid  1000
gravity**(1/4) 6.98 mns
grid  300
grid  500
grid  1000
Denver, United States done 7.34 mns
Ghent, Belgium
entrance 7.37 mns
grid  300
grid  500
grid  1000
gravity 9.31 mns
grid  300
grid  500
grid  1000
gravity**(1/2) 10.75 mns
grid  300
grid  500
grid  1000
gravity**(1/4) 12.04 mns
grid  300
grid  500
grid  1000
Ghent, Belgium done 13.3 mns
Amsterdam, Netherlands
entrance 13.31 mns
grid  300
grid  500
grid  1000
gravity 13.57 mns
grid  300
grid  500
grid  1000
gravity**(1/2) 13.82 mns
grid  300
grid  500
grid  1000
gravity**(1/4) 14.08 mns
gr

Philadelphia, United States  \
entrance_300        1 high                       0.003139   
                    2 medium                     0.073072   
                    3 low                        0.118656   
                    4 no                         0.805133   
entrance_500        1 high                       0.018444   
                    2 medium                     0.143630   
                    3 low                        0.223088   
                    4 no                         0.614839   
entrance_1000       1 high                       0.161281   
                    2 medium                     0.290134   
                    3 low                        0.294503   
                    4 no                         0.254081   
gravity_300         1 high                       0.001828   
                    2 medium                     0.136053   
                    3 low                        0.198409   
                    4 no                         0.663710   
gravity_500         1 high                       0.020743   
                    2 medium                     0.273683   
                    3 low                        0.303411   
                    4 no                         0.402163   
gravity_1000        1 high                       0.334311   
                    2 medium                     0.353187   
                    3 low                        0.204012   
                    4 no                         0.108491   
gravity**(1/2)_300  1 high                       0.000789   
                    2 medium                     0.083872   
                    3 low                        0.113457   
                    4 no                         0.801882   
gravity**(1/2)_500  1 high                       0.006484   
                    2 medium                     0.156421   
                    3 low                        0.207906   
                    4 no                         0.629189   
gravity**(1/2)_1000 1 high                       0.078525   
                    2 medium                     0.345504   
                    3 low                        0.300159   
                    4 no                         0.275812   
gravity**(1/4)_300  1 high                       0.000970   
                    2 medium                     0.074703   
                    3 low                        0.104377   
                    4 no                         0.819951   
gravity**(1/4)_500  1 high                       0.006765   
                    2 medium                     0.140326   
                    3 low                        0.190467   
                    4 no                         0.662441   
gravity**(1/4)_1000 1 high                       0.076342   
                    2 medium                     0.310069   
                    3 low                        0.316201   
                    4 no                         0.297388   

                              Denver, United States  Ghent, Belgium  \
entrance_300        1 high                 0.002564        0.001494   
                    2 medium               0.070264        0.043877   
                    3 low                  0.106396        0.071117   
                    4 no                   0.820776        0.883512   
entrance_500        1 high                 0.013865        0.012098   
                    2 medium               0.130575        0.086298   
                    3 low                  0.201593        0.117743   
                    4 no                   0.653968        0.783861   
entrance_1000       1 high                 0.093969        0.107673   
                    2 medium               0.296510        0.150541   
                    3 low                  0.298024        0.177610   
                    4 no                   0.311496        0.564176   
gravity_300         1 high                 0.003655        0.027130   
                    2 medium               0.1

In [28]:
popg_acc.to_csv(r'C:\Users\bartb\Downloads\popgrid_access.csv')

In [ ]:
grid_scores[0][0]

In [ ]:
# Use the parkgrid information from grid summarizing to get a summary per park.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)-1):
    print(cities[i])

    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav','grav2','grav4'])
    m1 = list(['entrance','gravity','gravity**(1/2)','gravity**(1/4)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'walkha_' + score]] = str1.groupby(
                'Park_No')[score,'pop' + score,'walk_area_ha' + str2].sum()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            prk = prk.fillna(-1)
            print(thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
        parks.append(parksc)
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        parksc.to_csv('C:/Users/bartb/Downloads/parkscore_'+ l1[i] + str2 + '.csv')

        print(l1[j])
    cityparks.append(parks)
pd.DataFrame(cityparks[1][1])

Philadelphia, United States
raw
grav
grav2
grav4
Denver, United States
raw
grav
grav2
grav4
Ghent, Belgium
raw


In [ ]:
# Need to adjust, get the preferred park for each grid
pref_parks = list([])
for n in range(len(cities)):
    l1 = list(['raw','grav','grav2','grav4'])
    m1 = list(['entrance','gravity','gravity**(1/2)','gravity**(1/4)'])
    prefer = list([])
    print(cities[n])
    for j in range(len(l1)):
        preference = list([])
        for k in range(len(thresholds)):
            t = str(thresholds[k])
            score = 'tr_' + t
            str1 = gridpark[n][j]
            pref = pd.DataFrame(str1.groupby('Grid_No')[score].idxmax().dropna().astype(int))
            pref = pd.merge(pref, gridpark[n][j][['Park_No','PoP2015_Number','pop'+score,'walk_area_ha'+t]], 
                            left_index = True, right_on = 'Park_No')
            pref = str1.loc[pref.iloc[:,0]]
            pref = pref[[score,'pop'+score,'walk_area_ha'+t,'geometry_x','Grid_No','Park_No','Parkroad_osmid','PoP2015_Number','geometry_y']]
            pref = pd.merge(pref, ParkRoads[n][['osmid','geometry_x']], 
                                        left_on = 'Parkroad_osmid', right_on = 'osmid', how = 'left')

            lines = list()
            for i in range(len(pref)):
                if pref['geometry_x_y'].iloc[i] != None:
                    lines.append(LineString([pref['geometry_y'].iloc[i].centroid, pref['geometry_x_y'].iloc[i]]))
                else: lines.append(None)
            pref['lines'] = lines
            preference.append(pref)
        print(l1[j])
        prefer.append(preference)
    pref_parks.append(prefer)
pd.DataFrame(pref_parks[0][0][0])

In [ ]:
pref

In [652]:
pref.columns

Index(['index', 'geometry_x', 'way_calculated', 'route_cost', 'num_steps',
       'Grid_No', 'Park_No', 'Park_entry_No', 'grid_entry_dist',
       'Parkroad_osmid', 'grid_osm', 'walk_area_m2', 'size_infl_factor',
       'size_infl_proot2', 'size_infl_proot4', 'raw euclidean',
       'raw_total_cost', 'parkcost_ha', 'gridpark_no', 'PoP2015_Number',
       'geometry_y', 'tr_300', 'poptr_300', 'walk_area_ha300', 'tr_500',
       'poptr_500', 'walk_area_ha500'],
      dtype='object')

In [ ]:
# To line files for visual inspection

gridpark_lines = gpd.GeoDataFrame(gridpark_grav2, geometry = 'geometry_x', crs = 4326)
gridpark_nodes = gpd.GeoDataFrame(gridpark_grav2, geometry = 'geometry_y', crs = 4326)

gridpark_lines[['geometry_x','score','Grid_No','Park_No','PoP2015_Number','pop_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_lines.shp')

gridpark_nodes[['geometry_y','score','Grid_No','Park_No','PoP2015_Number','pop_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_nodes.shp')

In [ ]:
grid_prefer = gridpark_grav2.iloc[gridpark_grav2.groupby('Grid_No')['score'].idxmax()]
grid_prefer = grid_prefer[['geometry_x','Parkroad_osmid','score','Grid_No','Park_No','geometry_y','pop_score']]
grid_prefer = pd.merge(grid_prefer, PtR[['osmid','geometry_x']], left_on = 'Parkroad_osmid', right_on = 'osmid', how = 'left')
lines = list()
for i in range(len(grid_prefer)):
    if grid_prefer['geometry_x_y'].iloc[i] != None:
        lines.append(LineString([grid_prefer['geometry_y'].iloc[i].centroid, grid_prefer['geometry_x_y'].iloc[i]]))
    else: lines.append(None)
grid_prefer['lines'] = lines
gpd.GeoDataFrame(grid_prefer[['Parkroad_osmid','score','Grid_No','Park_No','pop_score','lines']], 
                 geometry = 'lines', crs = 4326).to_file(r'C:\Users\bartb\Downloads\gridpark_preference.shp')
grid_prefer

In [ ]:
print(round((time.time() - start) / 60,2),'mns')